# Musicians - Easy

## Tables used in the musicians database

band(**band_no**, band_name, band_home, band_type, b_date, band_contact)

composer(**comp_no**, comp_is, comp_type)

composition(**c_no**, comp_date, c_title, c_in)

concert(**concert_no**, concert_venue, concert_in, con_date, concert_orgniser)

has_composed(**_cmpr_no_**, **_cmpn_no_**)

musician(**m_no**, m_name, born, died, born_in, living_in)

performance(**_pfrmnc_no_**, gave, performed, conducted_by, performed_in)

performer(**perf_no**, perf_is, instrument, perf_type)

place(**place_no**, place_town, place_country)

plays_in(**_player_**, **band_id**)

- **musician**

m_no | m_name | born | died | born_in | living_in
-:|-----------|------|------|--------:|-----:
1 | Fred Bloggs | 02/01/48 |  | 1 | 2
2 | John Smith | 03/03/50 |  | 3 | 4
3 | Helen Smyth | 08/08/48 |  | 4 | 5
4 | Harriet Smithson | 09/05/1909 | 20/09/80 | 5 | 6
5 | James First | 10/06/65 |  | 7 | 7
... | | | | | 

- **place**

place_no | place_town | place_country
--------:|------------|---------
1 | Manchester | England
2 | Edinburgh | Scotland
3 | Salzburg | Austria
4 | New York | USA
5 | Birmingham | England
... | | 

- **performer**

perf_no | perf_is | instrument | perf_type
-:|--:|--------|---------------------
1 | 2 | violin | classical
2 | 4 | viola | classical
3 | 6 | banjo | jazz
4 | 8 | violin | classical
5 | 12 | guitar | jazz
...| | |

- **composer**

comp_no | comp_is | comp_type
-:|--:|----------------
1 | 1 | jazz
2 | 3 | classical
3 | 5 | jazz
4 | 7 | classical
5 | 9 | jazz
... | |

- **band**

band_no | band_name | band_home | band_type | b_date | band_contact
-:|-----|--:|-----------|----------|----:
1 | ROP | 5 | classical | 30/01/01 | 11
2 | AASO | 6 | classical |  | 10
3 | The J Bs | 8 | jazz |  | 12
4 | BBSO | 9 | classical |  | 21
5 | The left Overs | 2 | jazz |  | 8
... | | | | |

- **plays_in**

player | band_id
-:|---:
1 | 1
1 | 7
3 | 1
4 | 1
4 | 7
... |

- **composition**

c_no | comp_date | c_title | c_in
-:|----------|--------|-------:
1 | 17/06/75 | Opus 1 | 1
2 | 21/07/76 | Here Goes | 2
3 | 14/12/81 | Valiant Knight | 3
4 | 12/01/82 | Little Piece | 4
5 | 13/03/85 | Simple Song | 5
... | | |

- **has_composed**

cmpr_no | cmpn_no
-:|---:
1 | 1
1 | 8
2 | 11
3 | 2
3 | 13
... |

- **concert**

concert_no | concert_venue | concert_in | con_date | concert_orgniser
-:|------------------|--:|----------|----:
1 | Bridgewater Hall | 1 | 06/01/95 | 21
2 | Bridgewater Hall | 1 | 08/05/96 | 3
3 | Usher Hall | 2 | 03/06/95 | 3
4 | Assembly Rooms | 2 | 20/09/97 | 21
5 | Festspiel Haus | 3 | 21/02/95 | 8
... | | | |

- **performance**

pfrmnc_no | gave | performed | conducted_by | performed_in
-:|--:|--:|---:|-----------------------:
1 | 1 | 1 | 21 | 1
2 | 1 | 3 | 21 | 1
3 | 1 | 5 | 21 | 1
4 | 1 | 2 | 1 | 2
5 | 2 | 4 | 21 | 2
... | | | |

Here is the data model of the musician tables

![](../src/img/musician_str.svg)

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app16-1")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@46c0cc87

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@74c4f249

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val composer = hiveCxt.table("sqlzoo.composer")
val composition = hiveCxt.table("sqlzoo.composition")
val concert = hiveCxt.table("sqlzoo.concert")
val has_composed = hiveCxt.table("sqlzoo.has_composed")
val musician = hiveCxt.table("sqlzoo.musician")
val performance = hiveCxt.table("sqlzoo.performance")
val performer = hiveCxt.table("sqlzoo.performer")
val place = hiveCxt.table("sqlzoo.place")
val plays_in = hiveCxt.table("sqlzoo.plays_in")

composer: DataFrame = [comp_no: int, comp_is: int ... 1 more field]
composition: DataFrame = [c_no: int, comp_date: string ... 2 more fields]
concert: DataFrame = [concert_no: int, concert_venue: string ... 3 more fields]
has_composed: DataFrame = [cmpr_no: int, cmpn_no: int]
musician: DataFrame = [m_no: int, m_name: string ... 4 more fields]
performance: DataFrame = [pfrmnc_no: int, gave: int ... 3 more fields]
performer: DataFrame = [perf_no: int, perf_is: int ... 2 more fields]
place: DataFrame = [place_no: int, place_town: string ... 1 more field]
plays_in: DataFrame = [player: int, band_id: int]

## 1.
**Give the organiser"s name of the concert in the Assembly Rooms after the first of Feb, 1997.**

In [5]:
(concert.filter((concert("concert_venue") === "Assembly Rooms") && 
                (concert("con_date")>"1997-02-01"))
 .join(musician, (concert("concert_orgniser") === musician("m_no")))
 .select("m_name")
 .showHTML())

m_name
James Steeple


## 2.
**Find all the performers who played guitar or violin and were born in England.**

In [6]:
(performer.filter(performer("instrument").isin(List("violin", "guitar"): _*))
 .join(musician, (performer("perf_is") === musician("m_no")))
 .join(place.filter(place("place_country") === "England"),
       (musician("born_in") === place("place_no")))
 .select("m_name")
 .orderBy("m_name")
 .showHTML())

m_name
Alan Fluff
Davis Heavan
Harriet Smithson
Harry Forte
James First
Theo Mengel


## 3.
**List the names of musicians who have conducted concerts in USA together with the towns and dates of these concerts.**

In [7]:
(concert.join(place.filter(place("place_country") === "USA"),
              (concert("concert_in") === place("place_no")))
 .join(performance, (concert("concert_no") === performance("performed_in")))
 .join(musician, (performance("conducted_by") === musician("m_no")))
 .select("m_name", "place_town", "con_date")
 .distinct()
 .orderBy("m_name")
 .showHTML())

m_name,place_town,con_date
James Steeple,New York,1997-06-15


## 4.
**How many concerts have featured at least one composition by Andy Jones? List concert date, venue and the composition's title.**

In [8]:
(concert.join(composition, (concert("concert_no") === composition("c_in")))
 .join(has_composed, (composition("c_no") === has_composed("cmpn_no")))
 .join(composer, (has_composed("cmpr_no") === composer("comp_no")))
 .join(musician.filter(musician("m_name") === "Andy Jones"), 
       (composer("comp_is") === musician("m_no")))
 .select("con_date", "concert_venue", "c_title")
 .showHTML())

con_date,concert_venue,c_title
1996-05-08,Bridgewater Hall,A Simple Piece


## 5.
**List the different instruments played by the musicians and avg number of musicians who play the instrument.**

In [9]:
val t = (performer.join(musician,
                        (performer("perf_is") === musician("m_no"))))
(t.groupBy("instrument")
 .pivot("m_name")
 .agg(count("m_no"))
 .union(t.withColumn("instrument", lit("ALL"))
        .groupBy("instrument")
        .pivot("m_name")
        .agg(count("m_no")))
 .join(t.groupBy("instrument")
       .agg(count("m_no").alias("ALL"))
       .union(t.withColumn("instrument", lit("ALL"))
              .groupBy("instrument")
              .agg(count("m_no").alias("ALL"))),
       "instrument")
 .orderBy("instrument")
 .na.fill(0)
 .showHTML())

instrument,Alan Fluff,Davis Heavan,Elsie James,Harriet Smithson,Harry Forte,Helen Smyth,James First,James Quick,Jeff Dawn,John Smith,Louise Simpson,Sue Little,Theo Mengel,ALL
ALL,2,3,3,2,3,1,1,2,2,3,3,1,3,29
banjo,0,0,0,0,0,0,0,0,0,0,0,0,1,1
bass,0,0,0,0,0,0,0,0,1,1,1,0,0,3
cello,0,1,1,0,0,0,0,0,0,0,0,1,0,3
clarinet,0,0,0,0,1,0,0,0,0,0,0,0,0,1
cornet,0,0,0,0,0,0,0,0,0,0,1,0,0,1
drums,0,0,0,0,1,0,0,0,0,0,0,0,1,2
flute,0,0,0,0,0,0,0,1,0,1,0,0,0,2
guitar,0,1,1,0,0,0,0,0,0,0,0,0,0,2
horn,0,0,0,0,0,1,0,0,0,0,0,0,0,1


t: DataFrame = [perf_no: int, perf_is: int ... 8 more fields]

In [10]:
spark.stop()